# Data Challenge

## Arturo Galvan-Alarcon

# GOAL: create a model to predict flight delays (ARR_DEL15 boolean target)

I will do three approaches:

* APPROACH 1 - ALL DATA
* APPROACH 2 - USEFUL FEATURES FROM COORELATION MATRIX
* APPROACH 3 - USEFUL FEATURES FROM FEATURE IMPORTANCE

In [1]:
import numpy as np
import pandas as pd

# df = pd.read_csv('aggregated.csv', nrows=100000)# 5 129 354
df = pd.read_csv('aggregated.csv')

In [2]:
date = df['FL_DATE'].str.split('-', n = 2, expand = True).astype('float64')
df['YEAR'] = date[0]
df['DAY'] = date[2]


df = df.replace([np.inf, -np.inf], np.nan)
df = df.drop(["Unnamed: 13", "FL_DATE"], axis=1).dropna()



df.head(5)

,MONTH,DAY_OF_WEEK,UNIQUE_CARRIER,FL_NUM,ORIGIN,ORIGIN_CITY_NAME,DEST,DEST_CITY_NAME,CRS_DEP_TIME,ARR_DEL15,CRS_ELAPSED_TIME,DISTANCE,YEAR,DAY
0,2.0,6.0,B6,28.0,MCO,"Orlando, FL",EWR,"Newark, NJ",1000.0,0.0,156.0,937.0,2017.0,25.0
1,2.0,7.0,B6,28.0,MCO,"Orlando, FL",EWR,"Newark, NJ",739.0,0.0,153.0,937.0,2017.0,26.0
2,2.0,1.0,B6,28.0,MCO,"Orlando, FL",EWR,"Newark, NJ",1028.0,0.0,158.0,937.0,2017.0,27.0
3,2.0,2.0,B6,28.0,MCO,"Orlando, FL",EWR,"Newark, NJ",739.0,0.0,153.0,937.0,2017.0,28.0
4,2.0,3.0,B6,33.0,BTV,"Burlington, VT",JFK,"New York, NY",1907.0,0.0,90.0,266.0,2017.0,1.0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5058334 entries, 0 to 5129353
Data columns (total 14 columns):
 #   Column            Dtype  
---  ------            -----  
 0   MONTH             float64
 1   DAY_OF_WEEK       float64
 2   UNIQUE_CARRIER    object 
 3   FL_NUM            float64
 4   ORIGIN            object 
 5   ORIGIN_CITY_NAME  object 
 6   DEST              object 
 7   DEST_CITY_NAME    object 
 8   CRS_DEP_TIME      float64
 9   ARR_DEL15         float64
 10  CRS_ELAPSED_TIME  float64
 11  DISTANCE          float64
 12  YEAR              float64
 13  DAY               float64
dtypes: float64(9), object(5)
memory usage: 578.9+ MB


## ENCODE columns with object values into numbers

In [4]:
def encode_labels(labels):
    from sklearn import preprocessing
    
    le = preprocessing.LabelEncoder()
    le.fit(labels)
    return le

In [5]:
carrier_encoder = encode_labels(df.UNIQUE_CARRIER)
df.UNIQUE_CARRIER = carrier_encoder.transform(df.UNIQUE_CARRIER).astype('float64')


origin_encoder = encode_labels(df.ORIGIN)
df.ORIGIN = origin_encoder.transform(df.ORIGIN).astype('float64')

origin_city_encoder = encode_labels(df.ORIGIN_CITY_NAME)
df.ORIGIN_CITY_NAME = origin_city_encoder.transform(df.ORIGIN_CITY_NAME).astype('float64')


dest_encoder = encode_labels(df.DEST)
df.DEST = dest_encoder.transform(df.DEST).astype('float64')

dest_city_encoder = encode_labels(df.DEST_CITY_NAME)
df.DEST_CITY_NAME = dest_city_encoder.transform(df.DEST_CITY_NAME).astype('float64')

In [6]:
master_df = df.copy()

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5058334 entries, 0 to 5129353
Data columns (total 14 columns):
 #   Column            Dtype  
---  ------            -----  
 0   MONTH             float64
 1   DAY_OF_WEEK       float64
 2   UNIQUE_CARRIER    float64
 3   FL_NUM            float64
 4   ORIGIN            float64
 5   ORIGIN_CITY_NAME  float64
 6   DEST              float64
 7   DEST_CITY_NAME    float64
 8   CRS_DEP_TIME      float64
 9   ARR_DEL15         float64
 10  CRS_ELAPSED_TIME  float64
 11  DISTANCE          float64
 12  YEAR              float64
 13  DAY               float64
dtypes: float64(14)
memory usage: 578.9 MB


## Model function

In [7]:
feature_importances = []

In [8]:
def XGB(train_x, train_y, test_x, test_y):
    import xgboost as xgb
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import confusion_matrix


    params = {
        'tree_method':"hist", 
        'objective':'reg:squarederror', 
        'learning_rate':0.01, 
    #     'early_stopping_rounds':6,
        'min_child_weight':0.5,
        'max_depth': 15
    }

    model = xgb.XGBClassifier(**params)
    model.fit(train_x, train_y)
    pred = model.predict(test_x)

    return accuracy_score(test_y,pred), confusion_matrix(test_y, pred), model

# APPROACH 1 - ALL DATA

In [9]:
target = df.ARR_DEL15
df = df.drop(["ARR_DEL15"], axis=1)

### TRAIN/TEST DATA

In [10]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y= train_test_split(df, target, test_size=0.15, random_state=42)

### XGBClassifier - all data

In [11]:
accuracy_1, matrix_1, model_1 = XGB(train_x, train_y, test_x, test_y)
print(accuracy_1)

0.8323613412041632


### Feature Importance

In [12]:
feature_importances_1 = []
feature_importances_1.append(model_1.feature_importances_)
feature_importances_1.append(train_x.columns)
feature_importances.append(feature_importances_1)
feature_importances_1

[array([0.2114379 , 0.05719128, 0.1515516 , 0.03127415, 0.05412694,
        0.06582016, 0.0554337 , 0.0775674 , 0.11127821, 0.04136954,
        0.04708241, 0.        , 0.09586672], dtype=float32),
 Index(['MONTH', 'DAY_OF_WEEK', 'UNIQUE_CARRIER', 'FL_NUM', 'ORIGIN',
        'ORIGIN_CITY_NAME', 'DEST', 'DEST_CITY_NAME', 'CRS_DEP_TIME',
        'CRS_ELAPSED_TIME', 'DISTANCE', 'YEAR', 'DAY'],
       dtype='object')]

# APPROACH 2 - CORRELATION MATRIX

In [13]:
# RESET DF FROM MASTER_DF
df = master_df.copy()

In [14]:
corr_matrix = df.corr()
corr_matrix['ARR_DEL15'].sort_values(ascending=False)

ARR_DEL15           1.000000
CRS_DEP_TIME        0.139923
DEST_CITY_NAME      0.033710
CRS_ELAPSED_TIME    0.028205
DISTANCE            0.026574
DEST                0.025279
YEAR                0.014014
UNIQUE_CARRIER      0.012475
ORIGIN              0.007675
ORIGIN_CITY_NAME    0.005284
DAY_OF_WEEK         0.003723
DAY                -0.000719
FL_NUM             -0.005766
MONTH              -0.011563
Name: ARR_DEL15, dtype: float64

### DROP any values that are not uesful according to the coorelation matrix


In [15]:
df = df.drop(["MONTH", 'FL_NUM', 'DAY'], axis=1).dropna() # tweaking


target = df.ARR_DEL15
df = df.drop(["ARR_DEL15"], axis=1)

### TRAIN/TEST DATA

In [16]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y= train_test_split(df, target, test_size=0.15, random_state=42)

### XGBClassifier - correlation matrix

In [17]:
accuracy_2, matrix_2, model_2 = XGB(train_x, train_y, test_x, test_y)
print(accuracy_2)

0.8205972710414879


### Feature Importance

In [18]:
feature_importances_2 = []
feature_importances_2.append(model_2.feature_importances_)
feature_importances_2.append(train_x.columns)
feature_importances.append(feature_importances_2)
feature_importances_2

[array([0.0289118 , 0.29406923, 0.05536764, 0.09881028, 0.07261533,
        0.13281745, 0.11489715, 0.04442807, 0.07868834, 0.07939468],
       dtype=float32),
 Index(['DAY_OF_WEEK', 'UNIQUE_CARRIER', 'ORIGIN', 'ORIGIN_CITY_NAME', 'DEST',
        'DEST_CITY_NAME', 'CRS_DEP_TIME', 'CRS_ELAPSED_TIME', 'DISTANCE',
        'YEAR'],
       dtype='object')]

# APPROACH 3 - Feature Importance 

In [34]:
# RESET DF FROM MASTER_DF
df = master_df.copy()

### DROP based on features

In [35]:
important_features_dict = {}
for x,i in enumerate(feature_importances_1[0]):
    important_features_dict[x]=i


important_features_list_1 = sorted(important_features_dict,
                                 key=important_features_dict.get,
                                 reverse=True)

print('Most important features: %s', important_features_list_1)

Most important features: %s [0, 2, 8, 12, 7, 5, 1, 6, 4, 10, 9, 3, 11]


In [36]:
important_features_dict = {}
for x,i in enumerate(feature_importances_2[0]):
    important_features_dict[x]=i


important_features_list_2 = sorted(important_features_dict,
                                 key=important_features_dict.get,
                                 reverse=True)

print('Most important features: %s', important_features_list_2)

Most important features: %s [1, 5, 6, 3, 9, 8, 4, 2, 7, 0]


In [37]:
for i in important_features_list_1:
    print(feature_importances_1[1][i])

MONTH
UNIQUE_CARRIER
CRS_DEP_TIME
DAY
DEST_CITY_NAME
ORIGIN_CITY_NAME
DAY_OF_WEEK
DEST
ORIGIN
DISTANCE
CRS_ELAPSED_TIME
FL_NUM
YEAR


In [38]:
for i in important_features_list_2:
    print(feature_importances_2[1][i])

UNIQUE_CARRIER
DEST_CITY_NAME
CRS_DEP_TIME
ORIGIN_CITY_NAME
YEAR
DISTANCE
DEST
ORIGIN
CRS_ELAPSED_TIME
DAY_OF_WEEK


In [39]:
df = df.drop(['YEAR', 'DAY_OF_WEEK', 'CRS_ELAPSED_TIME', 'FL_NUM', 'ORIGIN'], axis=1).dropna() # tweaking


target = df.ARR_DEL15
df = df.drop(["ARR_DEL15"], axis=1)

### TRAIN/TEST DATA

In [40]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y= train_test_split(df, target, test_size=0.15, random_state=42)

### XGBClassifier - Feature importance

In [41]:
accuracy_3, matrix_3, model_3 = XGB(train_x, train_y, test_x, test_y)
print(accuracy_3)

0.8316140604757029


### Feature Importance

In [42]:
feature_importances_3 = []
feature_importances_3.append(model_3.feature_importances_)
feature_importances_3.append(train_x.columns)
feature_importances.append(feature_importances_3)
feature_importances_3

[array([0.26957694, 0.18738425, 0.07413484, 0.07303786, 0.09943362,
        0.1222989 , 0.06016853, 0.11396504], dtype=float32),
 Index(['MONTH', 'UNIQUE_CARRIER', 'ORIGIN_CITY_NAME', 'DEST', 'DEST_CITY_NAME',
        'CRS_DEP_TIME', 'DISTANCE', 'DAY'],
       dtype='object')]

In [43]:
important_features_dict = {}
for x,i in enumerate(feature_importances_3[0]):
    important_features_dict[x]=i


important_features_list_3 = sorted(important_features_dict,
                                 key=important_features_dict.get,
                                 reverse=True)

print('Most important features: %s', important_features_list_3)

for i in important_features_list_3:
    print(feature_importances_3[1][i])

Most important features: %s [0, 1, 5, 7, 4, 2, 3, 6]
MONTH
UNIQUE_CARRIER
CRS_DEP_TIME
DAY
DEST_CITY_NAME
ORIGIN_CITY_NAME
DEST
DISTANCE


### Analysis of three different approaches

In [44]:
def printCM(accuracy, matrix):
#     import seaborn as sns

#     print("XGBClassifier")
    print("ACCURACY SCORE: ", accuracy)
#     print("\n\n")
    print("CONFUSION MATRIX: ")
    print(matrix)
#     sns.heatmap(matrix, fmt='.5g', annot=True)

In [45]:
printCM(accuracy_1, matrix_1)

ACCURACY SCORE:  0.8323613412041632
CONFUSION MATRIX: 
[[615666   6602]
 [120594  15889]]


In [46]:
printCM(accuracy_2, matrix_2)

ACCURACY SCORE:  0.8205972710414879
CONFUSION MATRIX: 
[[619033   3235]
 [132887   3596]]


In [47]:
printCM(accuracy_3, matrix_3)

ACCURACY SCORE:  0.8316140604757029
CONFUSION MATRIX: 
[[615613   6655]
 [121108  15375]]


# Conclusion - Approach 3 is the best with an accuracy of 83.16%

I did 3 appraches to achieve my goal, first I encoded all the data into numbers to feed it to my model without any issues.  I then made two new columns, "YEAR" and "DAY" to extract the information from the "FL_DATE" column.  After this, I proceeded to drop the "Unnamed: 13" and "FL_DATE" columns as they were useless to use in my model.  Once I completed cleaning my dataframe, I then applied my 3 methods on the data. 


I choose to use a classifier as my goal was to predict a boolean state as the outcome.  Selecting a classifier was a real issue as I initially tried Random Forest Trees Classification and this took too long for my computer to run. This lead me to googling what alternative classifiers existed for large datasets (5 Mil+) and found that an XGBClassifier was a good approach for me to use.  I tested out the performance between the two models with a sample size of 1000000 and got a better runtime and accuracy from the XGBClassifier.  Hence this became my model to use for the whole data challenge. 


The first approach was initially just to see how well the model performed without any modification, it was surprisingly scoring at a good accuracy.  I stored its feature importances into a list to later utilize.  My second appraoch was to use the correlation matrix output from the datafram for the target column "ARR_DEL15", this lead me to drop columns that gave a negative value or a NaN value.  After doing this, I reapplied my model and the accuracy was slightly worse than the first approach.  For this approach I also saved the feature importances into a new list.  Finally, my final apprached combined the first two approaches' importance features to get a dataframe that kept only the column/features that had the most significant influence in my model.  



Overall, my third and final approach scored a near same accuracy as my first approach however, I utilized a fraction of the columns/features in my final approach than in my first.  Additionally, viewing the confusion matrix I can also see that both approaches have a near same matrix with some slight variation.